<a href="https://colab.research.google.com/github/tomonari-masada/course-nlp2020/blob/master/07_document_classification_20201219.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 07 単語埋め込みを使った文書分類
* 今回も、IMDbデータセットの感情分析を文書分類問題として解く。
* ただし今回は、fastTextのような学習済みの単語埋め込みは使わない。
* 単語埋め込み自体の学習も、ネットワークの学習と同時におこなう。
* IMDbデータの準備も、`torch.torchtext`を使っておこなう。
 * つまりすべてをPyTorchのなかでおこなう。
* 参考資料
 * https://pytorch.org/tutorials/beginner/text_sentiment_ngrams_tutorial.html
 * https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/1%20-%20Simple%20Sentiment%20Analysis.ipynb
 * https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/4%20-%20Convolutional%20Sentiment%20Analysis.ipynb

## データをどう扱うか
* ネットワークへの入力は、単語埋め込みを、単語の出現順どおりに並べた列にする。
 * ミニバッチは[ミニバッチのなかでの最大文書長, ミニバッチのサイズ, 単語埋め込み次元数]という形の3階のテンソルになる。
* そして、前向き計算のなかではじめて、単語埋め込みの平均をとることにする。
 * `.mean(0)`と、軸0で平均をとることになる。


## 07-00 Google Colabのランタイムのタイプを変更する
* Google ColabのランタイムのタイプをGPUに変更しておこう。
 * 上のメニューの「ランタイム」→「ランタイムのタイプを変更」→「ハードウェア　アクセラレータ」から「GPU」を選択

## 07-01 torchtextを使ってIMDbデータを読み込む
* ここでIMDbデータセットの読み込みにつかう`torchtext.datasets`については、下記を参照。
 * https://torchtext.readthedocs.io/en/latest/datasets.html

### 実験の再現性確保のための設定など
* https://pytorch.org/docs/stable/notes/randomness.html

In [1]:
import random
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchtext import data
from torchtext import datasets
from torchtext.data import Field, LabelField, BucketIterator

SEED = 123

random.seed(SEED)
torch.manual_seed(SEED)
torch.set_deterministic(True)

### torchtextのフィールド
* TEXTフィールドと、LABELフィールドという２種類のFieldオブジェクトのインスタンスを作る。
 * Fieldクラスの詳細については[ここ](https://github.com/pytorch/text/blob/master/torchtext/data/field.py)を参照。
* TEXTフィールドは、テキストの前処理の仕方を決めておくのに使う。
 * tokenizerは、デフォルトでは単にstring型のsplitメソッドを適用するだけになる。これは高速だが、tokenizationとしては雑。
* LABELフィールドは、ラベルの前処理に使う。

In [2]:
TEXT = Field(tokenize="spacy")
LABEL = LabelField()

### IMDbデータセットをダウンロードした後、前処理しつつ読み込む
* ダウンロードはすぐ終わるが、解凍に少し時間がかかる。
* また、TEXTフィールドでspaCyのtokenizationを使うように設定したので、少し時間がかかる。
 * string型のsplitメソッドでtokenizeすると、時間はあまりかからない。（そのかわり、やや雑なtokenizationになる。）

In [3]:
train_valid_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

aclImdb_v1.tar.gz:   0%|          | 0.00/84.1M [00:00<?, ?B/s]

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:03<00:00, 23.2MB/s]


### 最初の文書を見てみる

In [4]:
print(train_valid_data[0].text)

['Not', 'sure', 'one', 'can', 'call', 'this', 'an', 'anti', '-', 'war', 'film', ',', 'it', 'shows', 'war', 'at', 'an', 'elite', 'level', '.', 'These', 'are', 'elite', 'troops', 'that', 'know', 'what', 'they', 'are', 'doing', 'and', 'take', 'great', 'pride', 'in', 'it', '.', 'Even', 'when', 'they', 'are', 'pacifist', ',', 'they', 'still', 'enjoy', 'the', 'skill', 'level', 'and', 'defeating', 'their', 'foes', ',', 'even', 'if', 'it', 'does', 'go', 'against', 'being', 'a', 'pacifist', '.', 'The', 'movies', 'is', 'slow', 'and', 'rather', 'uneventful', 'and', 'in', 'many', 'ways', 'is', 'rather', 'tame', 'as', 'war', 'movies', 'go', '-', 'more', 'so', 'by', 'todays', 'standards', ',', 'no', 'body', 'parts', 'flying', 'off', 'as', 'in', 'modern', 'movies', '.', 'It', 'is', 'brutal', 'in', 'other', 'ways', 'though', 'as', 'you', 'see', 'killing', 'at', 'a', 'personal', 'level', '.', 'This', 'is', 'more', 'of', 'a', 'thinking', 'man', "'s", 'movie', '.', 'Once', 'you', 'start', 'to', 'watch', 

In [5]:
print(train_valid_data[0].label)

pos


### テストセット以外の部分を訓練データと検証データに分ける

In [6]:
train_data, valid_data = train_valid_data.split(split_ratio=0.8)

In [7]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 20000
Number of validation examples: 5000
Number of testing examples: 25000


In [8]:
print(train_data[0].text)

['Ahh', ',', 'yes', ',', 'the', 'all', '-', 'star', 'blockbuster', '.', 'Take', 'a', 'so', '-', 'so', 'concept', ',', 'stuff', 'it', 'into', 'a', 'script', 'and', 'load', 'it', 'down', 'with', 'every', 'single', 'freakin', "'", 'special', 'effect', 'that', 'the', 'Wizards', 'of', 'Hollyweird', 'can', 'conjure', 'up', ',', 'then', 'round', 'up', 'the', 'usual', 'suspects', ':', 'hot', 'up', '-', 'and', '-', 'comers', ',', 'has', '-', 'beens', ',', 'wanna', '-', 'be', "'s", 'and', 'never', '-', 'wuzzes', ',', 'and', 'stick', "'em", 'all', 'in', 'ensemble', 'roles', 'of', 'various', 'sizes', 'in', 'front', 'of', 'the', 'unforgiving', 'eye', 'of', 'the', 'cameras', '.', 'And', 'hope', 'to', 'gawd', 'that', 'some', 'of', 'them', 'are', "n't", 'too', 'old', 'to', 'remember', 'their', 'lines.<br', '/><br', '/>Leave', 'it', 'to', 'the', 'bishops', 'of', 'Box', 'Office', 'to', 'apply', 'the', 'concept', 'to', 'horror', 'films', 'at', 'last', ',', 'as', 'was', 'the', 'case', 'with', 'the', 'post

### データセットの語彙とラベルを作る
* TEXTラベルのほうでは、最大語彙サイズを指定する。

In [9]:
MAX_VOCAB_SIZE = 25000 # この値は適当。

TEXT.build_vocab(train_data, max_size=MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

なぜ語彙サイズが25,000ではなく25,002なのかについては、少し下の説明を参照。

In [10]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2


### 出現頻度順で上位２０単語を見てみる

In [11]:
print(TEXT.vocab.freqs.most_common(20))

[('the', 231967), (',', 220822), ('.', 189768), ('a', 125308), ('and', 125211), ('of', 115201), ('to', 107546), ('is', 87468), ('in', 70357), ('I', 62046), ('it', 61426), ('that', 56383), ('"', 50715), ("'s", 49689), ('this', 48354), ('-', 42279), ('/><br', 40939), ('was', 40261), ('as', 34887), ('with', 34271)]


### 単語ID順に最初の１０単語を見てみる
* IDのうち、0と1は、未知語とパディング用の単語という特殊な単語に割り振られている。
 * 未知語は`<unk>`という特殊な単語に置き換えられる。これのIDが0。
 * パディングとは、長さが不揃いの複数の文書を同じミニバッチにまとめるとき、すべての文書の長さを無理やりそろえるため、文書末尾に特殊な単語（元々の語彙にない、人工的に用意した単語）を追加すること。
 * パディング用の単語が`<pad>`になっているのは、上のほうで使ったFieldクラスのインスタンスを作るときのデフォルトの値がこの`<pad>`になっているため。

In [12]:
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', 'the', ',', '.', 'a', 'and', 'of', 'to', 'is']


### ラベルのほうのIDを確認する
* こちらはnegとposに対応する２つのIDしかない。

In [13]:
print(LABEL.vocab.stoi)

defaultdict(<function _default_unk_index at 0x7fb4e9e62ea0>, {'neg': 0, 'pos': 1})


### ミニバッチを取り出すためのiteratorを作る
* ミニバッチのサイズを指定する。
 * ミニバッチのサイズは、性能を出すためにチューニングすべきハイパーパラメータのひとつ。

In [14]:
BATCH_SIZE = 100

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator = BucketIterator(train_data, batch_size=BATCH_SIZE, device=device,
                                     sort_within_batch=True, shuffle=True, sort_key=lambda x: len(x.text))
valid_iterator = BucketIterator(valid_data, batch_size=BATCH_SIZE, device=device)
test_iterator = BucketIterator(test_data, batch_size=BATCH_SIZE, device=device)

### ミニバッチの中身を確認する

* 訓練データのiteratorを回してミニバッチをすべて取得してみる
 * ミニバッチのshapeを表示してみる

In [15]:
for i, batch in enumerate(train_iterator):
  print(i, batch.text.shape)

0 torch.Size([133, 100])
1 torch.Size([455, 100])
2 torch.Size([251, 100])
3 torch.Size([213, 100])
4 torch.Size([106, 100])
5 torch.Size([381, 100])
6 torch.Size([94, 100])
7 torch.Size([187, 100])
8 torch.Size([289, 100])
9 torch.Size([113, 100])
10 torch.Size([49, 100])
11 torch.Size([536, 100])
12 torch.Size([189, 100])
13 torch.Size([162, 100])
14 torch.Size([228, 100])
15 torch.Size([135, 100])
16 torch.Size([390, 100])
17 torch.Size([834, 100])
18 torch.Size([182, 100])
19 torch.Size([170, 100])
20 torch.Size([245, 100])
21 torch.Size([934, 100])
22 torch.Size([1070, 100])
23 torch.Size([70, 100])
24 torch.Size([303, 100])
25 torch.Size([271, 100])
26 torch.Size([332, 100])
27 torch.Size([218, 100])
28 torch.Size([623, 100])
29 torch.Size([128, 100])
30 torch.Size([277, 100])
31 torch.Size([225, 100])
32 torch.Size([155, 100])
33 torch.Size([473, 100])
34 torch.Size([64, 100])
35 torch.Size([175, 100])
36 torch.Size([403, 100])
37 torch.Size([439, 100])
38 torch.Size([121, 100])

* ミニバッチの形は、[ミニバッチ内での最大文書長, ミニバッチのサイズ]になっていることに注意！
 * ミニバッチのサイズが最初に来ているのではない！
 * [ミニバッチのサイズ, ミニバッチ内での最大文書長]という形にしたいなら、テキストのfieldを作るとき以下のようにする。

__`TEXT = data.Field(tokenize="spacy", batch_first=True)`__

* 上記のループを抜けたあとには、変数batchには最後のミニバッチが代入されている。

In [16]:
batch.text.shape

torch.Size([139, 100])

* このミニバッチに含まれる文書のうち、最初の文書の単語ID列を表示させてみる。

In [17]:
print(batch.text[:, 0])

tensor([  744,    16,    24,     9,   654,  1108,    21,  4948,  3217,   154,
           34,     5,   308,    23,    12,    10,    84,   305, 10421,     6,
          154,    57,    29,   134,   190,     4,    11,   119,   154,    47,
           12,    99,     2,  1596,     7,     2,    77,     9,    50,   750,
            9,  1367,    48,   488,    12,   143,     6,   163,     6,    93,
            5,   139,    50,     4,   382,   173,     7,    31,    44,    57,
         1198,     8, 15563,     2,   759,   122,    31,   153,   380,   988,
            2,   302,     3,   113,   194,    47,     2,    24,    40,    42,
            4,   512,  2554,    12,    23,    31,    40,     2,   152,     7,
            2,    77,     5, 11962,  5466,  5493,   678,    64,  7521,     5,
         1787,    94,     2,  3662,  1505,     3,   925,   321,     3,   355,
         1068,     6, 16377,   207,     2,  3662,    40,     2,   189,    68,
            4,    11,   361,    12,   131,   175,   629,    26, 

* このミニバッチに含まれる文書のうち、最初の文書の単語ID列を単語列に戻したものを表示させてみる。

In [18]:
print(' '.join([TEXT.vocab.itos[i] for i in batch.text[:, 0]]))

Although this film is somewhat filled with eighties cheese i have a place for it in my DVD rack and i do n't know why . I think i like it because the moral of the story is ' television is garbage so turn it off and go and get a life ' . For those of you who do decide to heed the message then you should try reading the book , its nothing like the film at all . To ruin it for you at the end of the story a fatally wounded Richards ends up crashing a plane into the network building , killing himself , everyone inside and shutting down the network at the same time . I read it many years ago but today it would hard not to compare it to 9/11 .


* このミニバッチに含まれる文書のうち、最後の文書の単語ID列を表示させてみる。

In [19]:
print(batch.text[:, BATCH_SIZE-1])

tensor([  547,   203,    29,    16,   605,     7, 24728,     3,   472,   919,
           36,   872,    10,     2,   248,  1417,    58,    54,   953,     2,
          532,   579,     6,  3322,    42,     2,   110,     4,    25,   197,
            9,     5,   103,  3151,     4,  1024,  2307, 11955,   333,   447,
          525,     4,   866,   708,   188,   568,    37,    19, 21774,    23,
           35,   775,    20,  1945,  1409,     4,   155,     9,     5,  2071,
            8,   128,     4,   358,   108,    69,    81,   190,  6001,     0,
          104,    61,    88,  7340,   124,     4,   155,     9,     5,   103,
          748,   121,   298,     6,    35,   351,  1181,    21,  2307,     9,
          276,     8,   128,     4,  1094, 13993,    85,   791,   908,   179,
           10,     5,    33,    62, 18018,   237,     4,   400,   101,  1007,
        10606,    10,     8,   107,    16,     5,  1658,   797,   110,     8,
         1208,  6352,   258,     4,    11,  1810,   516, 12170, 

最後の文書の末尾は「1」で埋められていることが分かる。

この1は、パディング用単語のIDだったことを想起されたい。

ミニバッチに含まれる文書の長さを調べると、文書が文書長の降順に並べられていることが分かる。

In [20]:
(batch.text != 1).sum(0)

tensor([139, 139, 139, 139, 139, 139, 139, 139, 139, 139, 139, 139, 139, 139,
        139, 139, 139, 139, 139, 139, 139, 139, 139, 139, 139, 139, 139, 139,
        139, 139, 138, 138, 138, 138, 138, 138, 138, 138, 138, 138, 138, 138,
        138, 138, 138, 138, 138, 138, 138, 138, 138, 138, 138, 138, 138, 138,
        138, 138, 138, 138, 138, 138, 138, 138, 138, 138, 138, 138, 138, 138,
        138, 138, 138, 138, 138, 138, 138, 138, 138, 138, 138, 138, 138, 138,
        138, 137, 137, 137, 137, 137, 137, 137, 137, 137, 137, 137, 137, 137,
        137, 137], device='cuda:0')

## 07-02 MLPによる文書分類の準備
* 今回は、ごく簡単なMLPで文書分類をする。
* 文書中の全単語トークンの埋め込みベクトルの平均を、MLPの入力とする。
 * 当然、語順の情報は使われない。
 * つまり、bag-of-wordsモデルになっている。

### 定数の設定
* 単語埋め込みベクトルの次元数は128にする。

In [21]:
INPUT_DIM = len(TEXT.vocab)
NUM_CLASS = len(LABEL.vocab)
EMBED_DIM = 128
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

print(f'語彙サイズ {INPUT_DIM}, クラス数 {NUM_CLASS}, 単語埋め込み次元 {EMBED_DIM}')

語彙サイズ 25002, クラス数 2, 単語埋め込み次元 128


### モデルを定義する前にPyTorchの単語埋め込みがどんなものかを見てみる
* 埋め込みとは、単語IDから単語ベクトルへのマッピング。

* 以下のように、語彙サイズと埋め込みの次元数を指定しつつ、torch.nn.Embeddingのインスタンスを作ればよい。

In [22]:
embed = nn.Embedding(INPUT_DIM, EMBED_DIM, padding_idx=PAD_IDX).to(device)

* パディング用の単語の埋め込みはゼロベクトルになる。

In [23]:
print(embed(torch.tensor([21,1]).to(device)))

tensor([[ 1.4663,  0.3888,  0.5244, -0.6720,  1.2702,  0.1475, -0.6756,  1.0647,
          0.4122,  0.4360,  0.4711,  1.7838, -0.2890, -0.1943, -0.4321,  0.1552,
         -0.2901,  0.6956,  0.3870,  1.3461,  3.0132,  0.9346, -2.4039, -0.0521,
         -0.7123, -0.9902,  1.6379,  0.2446, -0.4733,  1.0840, -0.1715, -1.9957,
         -0.7140,  0.2289, -0.5881,  2.2581, -0.5802,  1.4646,  0.0392, -0.7295,
          0.4982, -1.1848,  0.4286, -0.0767,  1.2760, -1.3223,  0.0627,  0.3617,
         -0.7071,  0.0949, -0.5615, -0.6247,  1.0816,  0.7235, -0.9131,  0.6759,
          0.0999, -0.9366,  1.9773,  1.2333,  0.1287, -1.0535, -1.6376,  0.7970,
          0.1361, -0.0714, -0.2324, -0.8216,  1.4716, -0.0678,  0.4309, -0.1257,
          0.2348, -1.3550,  0.3339,  2.1095,  1.8534, -0.8028,  0.6104,  0.8844,
         -0.4362,  0.4587,  0.4177,  1.4428,  0.0063, -0.2002,  0.4557,  0.4221,
         -1.5692, -0.5961, -0.7103, -0.9230,  0.1227, -0.0557, -0.0169,  0.9816,
          0.1775,  0.8781, -

* 埋め込みの効果を見てみる

In [24]:
for i, batch in enumerate(train_iterator):
  print(i, embed(batch.text).shape)

0 torch.Size([404, 100, 128])
1 torch.Size([144, 100, 128])
2 torch.Size([282, 100, 128])
3 torch.Size([106, 100, 128])
4 torch.Size([209, 100, 128])
5 torch.Size([172, 100, 128])
6 torch.Size([515, 100, 128])
7 torch.Size([88, 100, 128])
8 torch.Size([82, 100, 128])
9 torch.Size([137, 100, 128])
10 torch.Size([231, 100, 128])
11 torch.Size([200, 100, 128])
12 torch.Size([351, 100, 128])
13 torch.Size([213, 100, 128])
14 torch.Size([747, 100, 128])
15 torch.Size([203, 100, 128])
16 torch.Size([916, 100, 128])
17 torch.Size([371, 100, 128])
18 torch.Size([149, 100, 128])
19 torch.Size([302, 100, 128])
20 torch.Size([589, 100, 128])
21 torch.Size([294, 100, 128])
22 torch.Size([197, 100, 128])
23 torch.Size([448, 100, 128])
24 torch.Size([621, 100, 128])
25 torch.Size([223, 100, 128])
26 torch.Size([64, 100, 128])
27 torch.Size([239, 100, 128])
28 torch.Size([823, 100, 128])
29 torch.Size([265, 100, 128])
30 torch.Size([220, 100, 128])
31 torch.Size([184, 100, 128])
32 torch.Size([194, 1

### モデルの定義
* MLP（多層パーセプトロン）だが、入り口に単語埋め込み層が挿入されている。

In [27]:
class EmbedTextSentiment(nn.Module):
  def __init__(self, embed_dim, num_class, vocab_size, padding_idx):
    super(EmbedTextSentiment, self).__init__()
    self.padding_idx = padding_idx # 2020/12/19追加
    self.embed = nn.Embedding(vocab_size, embed_dim, padding_idx=padding_idx)
    self.fc1 = nn.Linear(embed_dim, 500)
    self.fc2 = nn.Linear(500, 100)
    self.fc3 = nn.Linear(100, num_class)

  def forward(self, text):
    x = self.embed(text)
    #x = x.mean(0) # 文書に含まれる全単語トークンの単語ベクトルの平均
    #上の平均の計算を正確に書くと、以下の2行のようになります。（2020/12/19追加）
    mask = (text != self.padding_idx)
    x = (x * mask.unsqueeze(2)).sum(0) / mask.sum(0).unsqueeze(1)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

### モデルを作る
* モデル（のインスタンス）をGPUに移動させている点に注意。

In [28]:
model = EmbedTextSentiment(EMBED_DIM, NUM_CLASS, INPUT_DIM, padding_idx=PAD_IDX).to(device)

### 損失関数とoptimizerとschedulerを作る

In [29]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.95)

### 訓練用の関数
* 最初の`model.train()`に注意。こうやって、モデルを訓練モードに設定する。
 * 例えば、dropoutを含むモデルなど、訓練時と評価時で、ふるまい方を変える必要があるときがあるため、こういうことをする。

In [30]:
def train(data_iterator, model, optimizer, scheduler, criterion):

  model.train()

  train_loss = 0
  train_acc = 0
  for batch in data_iterator:
    optimizer.zero_grad()
    text, cls = batch.text, batch.label
    output = model(text)
    loss = criterion(output, cls)
    train_loss += loss.item()
    loss.backward()
    optimizer.step()
    train_acc += (output.argmax(1) == cls).float().mean().item()

  scheduler.step()

  num_batch = len(data_iterator)
  return train_loss / num_batch, train_acc / num_batch

### 評価用の関数
* 最初の`model.eval()`に注意。こうやって、モデルを評価モードに設定する。
 * 例えば、dropoutを含むモデルなど、訓練時と評価時で、ふるまい方を変える必要があるときがあるため、こういうことをする。

In [31]:
def test(data_iterator, model, criterion):

  model.eval()

  loss = 0
  acc = 0
  for batch in data_iterator:
    text, cls = batch.text, batch.label
    with torch.no_grad():
      output = model(text)
      loss = criterion(output, cls)
      loss += loss.item()
      acc += (output.argmax(1) == cls).float().mean().item()

  num_batch = len(data_iterator)
  return loss / num_batch, acc / num_batch

## 07-03 分類器の訓練と検証セットでの評価

In [32]:
N_EPOCHS = 20
for epoch in range(N_EPOCHS):

  start_time = time.time()
  train_loss, train_acc = train(train_iterator, model, optimizer, scheduler, criterion)
  valid_loss, valid_acc = test(valid_iterator, model, criterion)

  secs = int(time.time() - start_time)
  mins = secs // 60
  secs = secs % 60

  print(f'Epoch {epoch + 1} | time in {mins:d} minutes, {secs:d} seconds | ', end='')
  for param_group in optimizer.param_groups:
    print(f'lr={param_group["lr"]:.6f}')
    break
  print(f'\tLoss: {train_loss:.5f}(train)\t|\tAcc: {train_acc * 100:.2f}%(train)')
  print(f'\tLoss: {valid_loss:.5f}(valid)\t|\tAcc: {valid_acc * 100:.2f}%(valid)')

Epoch 1 | time in 0 minutes, 2 seconds | lr=0.009500
	Loss: 0.37151(train)	|	Acc: 83.24%(train)
	Loss: 0.01162(valid)	|	Acc: 88.08%(valid)
Epoch 2 | time in 0 minutes, 2 seconds | lr=0.009025
	Loss: 0.15511(train)	|	Acc: 94.36%(train)
	Loss: 0.01306(valid)	|	Acc: 87.58%(valid)
Epoch 3 | time in 0 minutes, 2 seconds | lr=0.008574
	Loss: 0.07948(train)	|	Acc: 97.24%(train)
	Loss: 0.00866(valid)	|	Acc: 87.48%(valid)
Epoch 4 | time in 0 minutes, 2 seconds | lr=0.008145
	Loss: 0.04204(train)	|	Acc: 98.50%(train)
	Loss: 0.02709(valid)	|	Acc: 86.52%(valid)
Epoch 5 | time in 0 minutes, 2 seconds | lr=0.007738
	Loss: 0.02673(train)	|	Acc: 99.06%(train)
	Loss: 0.01558(valid)	|	Acc: 86.86%(valid)
Epoch 6 | time in 0 minutes, 2 seconds | lr=0.007351
	Loss: 0.01378(train)	|	Acc: 99.52%(train)
	Loss: 0.01376(valid)	|	Acc: 87.06%(valid)
Epoch 7 | time in 0 minutes, 2 seconds | lr=0.006983
	Loss: 0.00854(train)	|	Acc: 99.69%(train)
	Loss: 0.05389(valid)	|	Acc: 86.06%(valid)
Epoch 8 | time in 0 minutes

## 07-04 再検討
* 訓練データ上での分類精度がほぼ100%になってしまっている。
* 検証データでの分類精度と大きな差があり、明らかにオーバーフィッティング。

### ドロップアウトを使う
* モデルのインスタンスを作るときにdropoutの確率を引数pで指定できるようにする。

In [36]:
class EmbedTextSentiment(nn.Module):
  def __init__(self, embed_dim, num_class, vocab_size, padding_idx, p=0.0):
    super(EmbedTextSentiment, self).__init__()
    self.padding_idx = padding_idx # 2020/12/19追加
    self.embed = nn.Embedding(vocab_size, embed_dim, padding_idx=padding_idx)
    self.dropout = nn.Dropout(p=p)
    self.fc1 = nn.Linear(embed_dim, 500)
    self.fc2 = nn.Linear(500, 100)
    self.fc3 = nn.Linear(100, num_class)

  def forward(self, text):
    x = self.dropout(self.embed(text)) #埋め込み層の直後にdropout
    #x = x.mean(0)
    #上の平均の計算を正確に書くと、以下の2行のようになります。（2020/12/19追加）
    mask = (text != self.padding_idx)
    x = (x * mask.unsqueeze(2)).sum(0) / mask.sum(0).unsqueeze(1)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

In [37]:
model = EmbedTextSentiment(EMBED_DIM, NUM_CLASS, INPUT_DIM, padding_idx=PAD_IDX, p=0.5).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.95)

In [38]:
N_EPOCHS = 20
for epoch in range(N_EPOCHS):

  start_time = time.time()
  train_loss, train_acc = train(train_iterator, model, optimizer, scheduler, criterion)
  valid_loss, valid_acc = test(valid_iterator, model, criterion)

  secs = int(time.time() - start_time)
  mins = secs // 60
  secs = secs % 60

  print(f'Epoch {epoch + 1} | time in {mins:d} minutes, {secs:d} seconds | ', end='')
  for param_group in optimizer.param_groups:
    print(f'lr={param_group["lr"]:.6f}')
    break
  print(f'\tLoss: {train_loss:.5f}(train)\t|\tAcc: {train_acc * 100:.2f}%(train)')
  print(f'\tLoss: {valid_loss:.5f}(valid)\t|\tAcc: {valid_acc * 100:.2f}%(valid)')

Epoch 1 | time in 0 minutes, 2 seconds | lr=0.009500
	Loss: 0.42817(train)	|	Acc: 79.67%(train)
	Loss: 0.01726(valid)	|	Acc: 88.28%(valid)
Epoch 2 | time in 0 minutes, 2 seconds | lr=0.009025
	Loss: 0.21941(train)	|	Acc: 91.53%(train)
	Loss: 0.00776(valid)	|	Acc: 88.22%(valid)
Epoch 3 | time in 0 minutes, 2 seconds | lr=0.008574
	Loss: 0.14934(train)	|	Acc: 94.38%(train)
	Loss: 0.01548(valid)	|	Acc: 88.48%(valid)
Epoch 4 | time in 0 minutes, 2 seconds | lr=0.008145
	Loss: 0.10527(train)	|	Acc: 96.12%(train)
	Loss: 0.01569(valid)	|	Acc: 88.24%(valid)
Epoch 5 | time in 0 minutes, 2 seconds | lr=0.007738
	Loss: 0.07221(train)	|	Acc: 97.24%(train)
	Loss: 0.01256(valid)	|	Acc: 88.18%(valid)
Epoch 6 | time in 0 minutes, 2 seconds | lr=0.007351
	Loss: 0.05581(train)	|	Acc: 98.00%(train)
	Loss: 0.02577(valid)	|	Acc: 87.58%(valid)
Epoch 7 | time in 0 minutes, 2 seconds | lr=0.006983
	Loss: 0.04255(train)	|	Acc: 98.41%(train)
	Loss: 0.02043(valid)	|	Acc: 87.94%(valid)
Epoch 8 | time in 0 minutes

### L２正則化を使う
* optimizerのweight_decayパラメータを0より大きな値にする。

In [39]:
model = EmbedTextSentiment(EMBED_DIM, NUM_CLASS, INPUT_DIM, padding_idx=PAD_IDX, p=0.5).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.95)

In [40]:
N_EPOCHS = 20
for epoch in range(N_EPOCHS):

  start_time = time.time()
  train_loss, train_acc = train(train_iterator, model, optimizer, scheduler, criterion)
  valid_loss, valid_acc = test(valid_iterator, model, criterion)

  secs = int(time.time() - start_time)
  mins = secs // 60
  secs = secs % 60

  print(f'Epoch {epoch + 1} | time in {mins:d} minutes, {secs:d} seconds | ', end='')
  for param_group in optimizer.param_groups:
    print(f'lr={param_group["lr"]:.6f}')
    break
  print(f'\tLoss: {train_loss:.5f}(train)\t|\tAcc: {train_acc * 100:.2f}%(train)')
  print(f'\tLoss: {valid_loss:.5f}(valid)\t|\tAcc: {valid_acc * 100:.2f}%(valid)')

Epoch 1 | time in 0 minutes, 2 seconds | lr=0.009500
	Loss: 0.57730(train)	|	Acc: 69.35%(train)
	Loss: 0.01858(valid)	|	Acc: 75.04%(valid)
Epoch 2 | time in 0 minutes, 2 seconds | lr=0.009025
	Loss: 0.44230(train)	|	Acc: 79.48%(train)
	Loss: 0.01357(valid)	|	Acc: 82.70%(valid)
Epoch 3 | time in 0 minutes, 2 seconds | lr=0.008574
	Loss: 0.38614(train)	|	Acc: 83.05%(train)
	Loss: 0.01318(valid)	|	Acc: 84.78%(valid)
Epoch 4 | time in 0 minutes, 2 seconds | lr=0.008145
	Loss: 0.36366(train)	|	Acc: 84.40%(train)
	Loss: 0.01520(valid)	|	Acc: 86.22%(valid)
Epoch 5 | time in 0 minutes, 2 seconds | lr=0.007738
	Loss: 0.33946(train)	|	Acc: 85.94%(train)
	Loss: 0.01635(valid)	|	Acc: 86.66%(valid)
Epoch 6 | time in 0 minutes, 2 seconds | lr=0.007351
	Loss: 0.32933(train)	|	Acc: 86.28%(train)
	Loss: 0.01078(valid)	|	Acc: 86.84%(valid)
Epoch 7 | time in 0 minutes, 2 seconds | lr=0.006983
	Loss: 0.32275(train)	|	Acc: 86.46%(train)
	Loss: 0.01018(valid)	|	Acc: 86.40%(valid)
Epoch 8 | time in 0 minutes

### early stopping
* validation setでのaccuracyが4回連続で最高値を下回ったら訓練を終えることにする。
* early stoppingの実現については、PyTorch Lightningを使う手もある。
 * https://pytorch-lightning.readthedocs.io/en/latest/early_stopping.html

In [41]:
model = EmbedTextSentiment(EMBED_DIM, NUM_CLASS, INPUT_DIM, padding_idx=PAD_IDX, p=0.5).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.95)

In [42]:
patience = 4
early_stop_count = 0
best_valid_acc = 0.0

MIN_N_EPOCHS = 10 # 最低このエポック数は実行する
N_EPOCHS = 50 # エポック数を増やしておく
for epoch in range(N_EPOCHS):

  start_time = time.time()
  train_loss, train_acc = train(train_iterator, model, optimizer, scheduler, criterion)
  valid_loss, valid_acc = test(valid_iterator, model, criterion)

  secs = int(time.time() - start_time)
  mins = secs // 60
  secs = secs % 60

  print(f'Epoch {epoch + 1} | time in {mins:d} minutes, {secs:d} seconds | ', end='')
  for param_group in optimizer.param_groups:
    print(f'lr={param_group["lr"]:.6f}')
    break
  print(f'\tLoss: {train_loss:.5f}(train)\t|\tAcc: {train_acc * 100:.2f}%(train)')
  print(f'\tLoss: {valid_loss:.5f}(valid)\t|\tAcc: {valid_acc * 100:.2f}%(valid)')

  # early stopping
  if epoch + 1 > MIN_N_EPOCHS:
    if best_valid_acc <= valid_acc:
      best_valid_acc = valid_acc
      early_stop_count = 0
    else:
      early_stop_count += 1
      if early_stop_count == patience:
        break

Epoch 1 | time in 0 minutes, 2 seconds | lr=0.009500
	Loss: 0.58522(train)	|	Acc: 68.47%(train)
	Loss: 0.02286(valid)	|	Acc: 75.24%(valid)
Epoch 2 | time in 0 minutes, 2 seconds | lr=0.009025
	Loss: 0.46134(train)	|	Acc: 78.34%(train)
	Loss: 0.01269(valid)	|	Acc: 84.08%(valid)
Epoch 3 | time in 0 minutes, 2 seconds | lr=0.008574
	Loss: 0.39208(train)	|	Acc: 82.79%(train)
	Loss: 0.01358(valid)	|	Acc: 84.64%(valid)
Epoch 4 | time in 0 minutes, 2 seconds | lr=0.008145
	Loss: 0.35059(train)	|	Acc: 84.91%(train)
	Loss: 0.01191(valid)	|	Acc: 86.76%(valid)
Epoch 5 | time in 0 minutes, 2 seconds | lr=0.007738
	Loss: 0.34777(train)	|	Acc: 85.09%(train)
	Loss: 0.01754(valid)	|	Acc: 86.82%(valid)
Epoch 6 | time in 0 minutes, 2 seconds | lr=0.007351
	Loss: 0.32731(train)	|	Acc: 86.26%(train)
	Loss: 0.01219(valid)	|	Acc: 86.92%(valid)
Epoch 7 | time in 0 minutes, 2 seconds | lr=0.006983
	Loss: 0.31185(train)	|	Acc: 87.31%(train)
	Loss: 0.01402(valid)	|	Acc: 85.74%(valid)
Epoch 8 | time in 0 minutes

## 07-05 テストセット上で評価
* 見つけ出したベストな設定を使って、テストセット上での最終的な評価をおこなう。

In [43]:
print('Checking the results of test dataset...')
test_loss, test_acc = test(test_iterator, model, criterion)
print(f'\tLoss: {test_loss:.5f}(test)\t|\tAcc: {test_acc * 100:.2f}%(test)')

Checking the results of test dataset...
	Loss: 0.00203(test)	|	Acc: 87.52%(test)
